<a href="https://colab.research.google.com/github/LCG17370228702/Xiaoxiang_Project/blob/main/xiaoxiang_nlp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/XiaoXiang_project')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

train_df = pd.read_csv('./data/train.csv', sep="###__###", header=None, names=['ID', 'Age', 'Gender', 'Education', 'Query_List'])
test_df = pd.read_csv('./data/test.csv', sep="###__###", header=None, names=['ID', 'Query_List'])
stopwords_df = pd.read_table('./data/cn_stopwords.txt', header=None, names=['StopWords'])
stopwords = stopwords_df['StopWords'].values

In [3]:
import re

def find_chinese(file):
  pattern = re.compile(r'[^\u4e00-\u9fa5 \t]')
  chinese_txt = re.sub(pattern,'',file)
  return chinese_txt

In [4]:
import jieba

train_age_df = train_df[['Age', 'Query_List']]
train_age_df = train_age_df[~(train_age_df['Age']==0)]  # 对缺失值直接进行删除
train_age_df['Query_List'] = train_age_df['Query_List'].apply(lambda x: find_chinese(x))
train_age_df['Query_List'] = train_age_df['Query_List'].apply(lambda x: jieba.lcut(x))
train_age_df['Query_List'] = train_age_df['Query_List'].apply(lambda x: filter(lambda y: len(y)>1, x))
train_age_df['Query_List'] = train_age_df['Query_List'].apply(lambda x: filter(lambda y: y not in stopwords, x))
train_age_df['Query_List'] = train_age_df['Query_List'].apply(lambda x: ' '.join(x))

train_age_df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built successfully.


,Age,Query_List
0,1,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 曹云金 再讽 郭德纲 总裁 大...
1,2,广州 厨宝 烤箱 世情 人情 恶雨 黄昏 花易 落晓 风干 泪痕 厦门 酒店用品 批发市场 ...
2,4,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 盘锦 台安 网游 永生 网游 辽...
3,4,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 一起 壮观 衣服 网站 动漫 绘图...
4,2,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 下面 一直 怎么回事 麦当劳 旋风 勺子...


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=4000).fit(train_age_df['Query_List'].iloc[:].values)
train_tfidf = tfidf.transform(train_age_df['Query_List'].iloc[:].values)
test_tfidf = tfidf.transform(test_df['Query_List'].iloc[:].values)

skf = StratifiedKFold(n_splits=5, random_state=42) 
test_pred = np.zeros((test_tfidf.shape[0], 6), dtype=np.float32)

for idx, (train_index, valid_index) in enumerate(skf.split(train_tfidf, train_age_df['Age'].values)):
  x_train_, x_valid_ = train_tfidf[train_index], train_tfidf[valid_index]
  y_train_, y_valid_ = train_age_df['Age'].values[train_index], train_age_df['Age'].values[valid_index]
  
  clf = LGBMClassifier()
  clf.fit(x_train_, y_train_)
  val_pred = clf.predict(x_valid_)
  
  print(f1_score(y_valid_, val_pred, average='macro'))
  test_pred += clf.predict_proba(test_tfidf)

0.3312084201204833
0.3385144209214305
0.33331765721245427
0.3368186381370939
0.337591561830142
